Trades
+ Адрес хранилища данных: points
Получение данных
Адрес хранилища данных: данные
+ download data
+ read file
Получение словаря датафремов трейдов
+ Данные: словарь датафреймов
+ method(type): type
Преобразования к таймфрему оси чарта
+ Датафрейм: Агрегированный датафрейм по таймфрему
+ Агрегация экзекьюшинов с разницей времени < 5s
+ Агрегация по таймфреймам
Рисование экзекьюшина на чарте
+ Dataframe: chart
+ plotly
главная программа
"Перенесение трейдов на график"
главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)
Рисование чарта
Рисование трейдов
Candlestick
Хранилище данных: chart object
подготовка данных для графика
+ адрес хранилища данных: строка
Получение данных
+ адресс файла (строка): содержимое файла
+ чтение данных
Подготовка датафрейма
Объект данных: строка
+ csv to Dataframe
+ Convert Data
+ Prepare columns
Выбор времени
Датафрейм : рабочий датафрейм
+ обрезаем датафрейм до нужных данных
Получение таймфрейма
+ Dataframe : Dataframe
+ трансформация датафрейма 
   в нужный таймфрейм
Plotting
+ Dataframe: Plott
ploly candlestick
Чарт


деф главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)

Неправильно работает группировка, так как в Qty неверно отображается объем

In [1]:
import pandas as pd
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
import win32com.client

plotly.tools.set_credentials_file(username='mkf', api_key='crwJntc1jJsAdaHKiDPL')

In [2]:
def fix_excel(trades_file):
    import win32com.client

    xl=win32com.client.Dispatch("Excel.Application")
    xl.Application.DisplayAlerts = False # disables Excel pop up message (for saving the file)
    wb = xl.Workbooks.Open(Filename=file_excel)
    wb.SaveAs(file_excel)
    wb.Close
    xl.Application.DisplayAlerts = True

In [3]:
def open_excel(path):
    try:
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    except CompDocError:
        fix_excel(path)
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    return open_file

In [4]:
def normalise_formats(storage_of_executions):
    important_columns = ['Order Id', 'Time', 'B/S', 'Qty', 'Price', 'Gross', 'Net', 'Route']
    copy_storage_of_executions = storage_of_executions.copy()
    for date, tickers in storage_of_executions.items():
        for ticker, executions in tickers.items():
            new_df = executions[important_columns]
            new_df.insert(0, 'Date_Time', pd.to_datetime((date + ' ' + new_df['Time']), dayfirst=False))
            new_df.insert(1, 'Ticker', ticker)
            new_df['BP_used'] = new_df['Price'] * new_df['Qty']
            new_df['Qty'] = new_df.apply(lambda x: x['Qty'] if new_df['B/S'][x.name] in ['B', 'C'] else -x['Qty'], axis=1)
            #new_df.insert(6, 'Pos_size', new_df['Qty'].cumsum())
            new_df.reset_index(inplace=True)
            copy_storage_of_executions[date][ticker] = group_by_order_final(new_df)
    return copy_storage_of_executions

In [17]:
def group_by_order(executions):
    executions_by_order = executions.groupby('Order Id').agg(({'index': 'first', 'Ticker' : 'first','Time' : 'first', 
                                                               'Date_Time' : 'first', 'B/S' : 'first', 'Qty' : 'sum',
                                                               #'Pos_size' : 'sum', 
                                                               'Gross' : 'sum', 'Route' : 'first',
                                                               'BP_used': 'sum'}))
    executions_by_order['Price'] = round(executions_by_order['BP_used'] / executions_by_order['Qty'], 3)
    return executions_by_order

In [19]:
def group_by_order_final(execution):
    new_group_by_orger = group_by_order(pd.DataFrame(columns=execution.columns))
    one_group = pd.DataFrame(columns=execution.columns)
    for i in range(len(execution)):
        if one_group.empty:
            one_group = one_group.append(execution.iloc[i, :])
        if i == len(execution) - 1:
            new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
            one_group = pd.DataFrame(columns=execution.columns)
        else:
            try:
                if execution['Order Id'][i] == execution['Order Id'][i+1]:
                    one_group = one_group.append(execution.iloc[i, :])
                else:
                    new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                    one_group = pd.DataFrame(columns=execution.columns)
            except:
                new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                one_group = pd.DataFrame(columns=execution.columns)
    new_group_by_orger.insert(6, 'Pos_size', new_group_by_orger['Qty'].cumsum())
    return new_group_by_orger

In [16]:
def preprocessing_trades(storage_of_executions):
    file_parsing = {}
    excel_df = storage_of_executions
    reindex = {a : b for a, b in zip(range(22), excel_df.iloc[2, :])}
    excel_df = excel_df.rename(columns=reindex)
    for i in range(len(excel_df)):
        if pd.notna(excel_df.iloc[i][0]):
            if re.match(r'\d{1,2}/\d{1,2}/\d{4}', excel_df.iloc[i][0]):
                date = excel_df.iloc[i][0]
                file_parsing[date] = {}
                continue

            elif re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0]):
                ticker = re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0])[1]
                file_parsing[date][ticker] = pd.DataFrame()
                continue

            elif excel_df.iloc[i][0] == 'Time':

                file_parsing[date][ticker].reindex(columns=excel_df.iloc[i])
                columns = excel_df.iloc[i]
                continue

            elif re.match(r'\d\d:\d\d:\d\d', excel_df.iloc[i][0]):
                file_parsing[date][ticker] = file_parsing[date][ticker].append(excel_df.iloc[i, :])
    date_ticker_exec = normalise_formats(file_parsing)
    return date_ticker_exec

In [ ]:
def create_points(data):
    days_trades = preprocessing_trades(file_name)
 

In [4]:
def make_chart_df(chart_file):
    pass

In [69]:
def create_chart_df(chart_file):
    chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
    chart_df[['Open', 'High', 'Low', 'Close', 'PC']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC']].apply(lambda x:  round(x, 2))
    chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))
    return chart_df

In [6]:
def draw_trades(trades_df):
    pass

In [7]:
def create_picture(chart_df, trades_df):
    create_chart(chart_df)
    draw_trades(trades_df)

In [97]:
trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'


In [15]:
if __name__ == '__main__':
    trades_df = processing_trades(trades_file)
    chart_df = make_chart_df(chart_file)
    create_picture(chart_df, trades_df)

In [64]:
chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
chart_df[['Open', 'High', 'Low', 'Close', 'PC']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC']].apply(lambda x:  round(x, 2))
chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))

In [66]:
chart_df.head()

,Date_Time,Date,Time,Open,High,Low,Close,PC
0,2016-07-15 15:55:00,15.07.2016,15:55:00,215.80,215.96,215.79,215.81,216.12
1,2016-07-15 15:50:00,15.07.2016,15:50:00,215.84,215.90,215.79,215.80,216.12
2,2016-07-15 15:45:00,15.07.2016,15:45:00,215.89,215.94,215.83,215.83,216.12
3,2016-07-15 15:40:00,15.07.2016,15:40:00,216.04,216.04,215.88,215.90,216.12
4,2016-07-15 15:35:00,15.07.2016,15:35:00,216.06,216.09,215.98,216.03,216.12


In [75]:
trace = go.Candlestick(x=chart_df.Date_Time,
                       open=chart_df.Open,
                       high=chart_df.High,
                       low=chart_df.Low,
                       close=chart_df.Close)
data = [trace]
#py.iplot(data)

In [91]:
excel_df = pd.read_excel(r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls', encoding='utf-16', header=None)

In [49]:
def transform_one_df_to_date_axis(date, ticker, detailed_df):
    #important_columns = ['Time', 'B/S', 'Qty', 'Price', 'Gross', 'Net', 'Order Id', 'Route']
    executions = detailed_df[date][ticker][important_columns]
    
    


def transform_to_date_axis(file_parsing, tickers=None):
    if not tickers:
        
        days_trades = preprocessing_trades(file_name)
    


In [20]:
%%time
date = '8/6/2018'
ticker = 'ARLO'

trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

opened_file = open_excel(trades_file)
date_tickers_exec = preprocessing_trades(opened_file)

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Wall time: 8.3 s


In [21]:
date_tickers_exec[date][ticker]

,index,Ticker,Time,Date_Time,B/S,Qty,Pos_size,Gross,Route,BP_used,Price
Order Id,,,,,,,,,,,
121819,3,ARLO,09:22:20,2018-08-06 09:22:20,B,176.0,176.0,0.00,CROX,3967.04,22.540
123167,5,ARLO,09:24:03,2018-08-06 09:24:03,B,100.0,276.0,0.00,ARCA,2231.00,22.310
128583,6,ARLO,09:31:18,2018-08-06 09:31:18,B,100.0,376.0,0.00,NYSE,2155.00,21.550
129324,7,ARLO,09:31:41,2018-08-06 09:31:41,S,-200.0,176.0,-233.76,EDGX,4272.00,-21.360
128682,8,ARLO,09:31:47,2018-08-06 09:31:47,S,-200.0,-24.0,-286.00,EDGX,4176.00,-20.880
137504,10,ARLO,09:54:53,2018-08-06 09:54:53,S,-200.0,-224.0,0.00,ARCA,4158.00,-20.790
138844,11,ARLO,10:01:10,2018-08-06 10:01:10,S,-100.0,-324.0,0.00,ARCA,2087.50,-20.875
138963,12,ARLO,10:27:30,2018-08-06 10:27:30,B,500.0,176.0,-381.50,EDGX,10785.00,21.570


In [22]:
date_tickers_exec['8/6/2018']['BRK/B']

,index,Ticker,Time,Date_Time,B/S,Qty,Pos_size,Gross,Route,BP_used,Price
Order Id,,,,,,,,,,,
105923,21,BRK/B,09:00:43,2018-08-06 09:00:43,T,-200.0,-200.0,0.00,ARCA,40970.00,-204.850
109207,22,BRK/B,09:04:23,2018-08-06 09:04:23,T,-100.0,-300.0,0.00,ARCA,20460.00,-204.600
117581,23,BRK/B,09:17:25,2018-08-06 09:17:25,B,100.0,-200.0,-5.00,ARCA,20490.00,204.900
117831,24,BRK/B,09:17:36,2018-08-06 09:17:36,B,200.0,0.0,-26.00,ARCA,40996.00,204.980
133891,26,BRK/B,09:41:18,2018-08-06 09:41:18,B,100.0,100.0,0.00,ARCA,20592.00,205.920
136030,27,BRK/B,09:48:26,2018-08-06 09:48:26,S,-100.0,0.0,168.00,ARCA,20760.00,-207.600
136133,28,BRK/B,09:52:04,2018-08-06 09:52:04,B,50.0,50.0,0.00,CROX,10367.75,207.355
137779,29,BRK/B,09:55:57,2018-08-06 09:55:57,B,100.0,150.0,0.00,ARCA,20664.00,206.640
137815,30,BRK/B,09:57:14,2018-08-06 09:57:14,S,-89.0,61.0,-67.12,EDGX,18369.60,-206.400


In [258]:
execution = date_tickers_exec['8/6/2018']['ARLO']#['Time'].apply(lambda x: dt.datetime.strptime(x, '%H:%M:%S')

In [200]:
#group_by_order(pd.DataFrame(columns=execution.columns)).append(executions_by_order)

In [287]:
execution
#execution['Qty'] = [execution['Qty'] if execution['B/S'] in ['B','C'] else -execution['Qty']]
execution['Qty'] = execution.apply(lambda x: x['Qty'] if execution['B/S'][x.name] in ['B', 'C'] else -x['Qty'], axis=1)
execution['Pos_side'] = execution['Qty'].cumsum()

In [288]:
execution

,index,Date_Time,Ticker,Order Id,Time,B/S,Qty,Price,Gross,Net,Route,BP_used,Pos_side
0,3,2018-08-06 09:22:20,ARLO,121819,09:22:20,B,88.0,22.540,0.00,-0.626,CROX,1983.52,88.0
1,4,2018-08-06 09:22:20,ARLO,121819,09:22:20,B,112.0,22.520,0.00,-0.850,ARCA,2522.24,200.0
2,5,2018-08-06 09:24:03,ARLO,123167,09:24:03,B,100.0,22.310,0.00,-0.760,ARCA,2231.00,300.0
3,6,2018-08-06 09:31:18,ARLO,128583,09:31:18,B,100.0,21.550,0.00,-0.560,NYSE,2155.00,400.0
4,7,2018-08-06 09:31:41,ARLO,129324,09:31:41,S,-200.0,21.360,-233.76,-235.360,EDGX,4272.00,200.0
5,8,2018-08-06 09:31:47,ARLO,128682,09:31:47,S,-100.0,20.880,-143.00,-143.810,EDGX,2088.00,100.0
6,9,2018-08-06 09:31:47,ARLO,128682,09:31:47,S,-100.0,20.890,-66.00,-66.810,EDGX,2089.00,0.0
7,10,2018-08-06 09:54:53,ARLO,137504,09:54:53,S,-200.0,20.790,0.00,-1.600,ARCA,4158.00,-200.0
8,11,2018-08-06 10:01:10,ARLO,138844,10:01:10,S,-100.0,20.875,0.00,-0.810,ARCA,2087.50,-300.0
9,12,2018-08-06 10:27:30,ARLO,138963,10:27:30,B,250.0,21.570,-190.75,-192.635,EDGX,5392.50,-50.0


In [41]:
new_group_by_orger

NameError: name 'new_group_by_orger' is not defined

In [63]:
date_of_trade.date()

datetime.date(2018, 8, 6)

In [75]:
drowing_trades = date_tickers_exec['8/6/2018']['ARLO']

In [79]:
drowing_trades.apply(lambda x: x.replace(second=0))

TypeError: ("replace() got an unexpected keyword argument 'second'", 'occurred at index index')

In [52]:
b = a.replace(day=a.day+1)

In [46]:
a = date_tickers_exec[date]['ARLO'].Date_Time[0].date()

In [70]:
chart_df = create_chart_df(r'C:\Users\User\Documents\ARLO_06.08.18.csv')

In [73]:
date_of_trade = date_tickers_exec[date]['ARLO'].Date_Time[0]
needed_chart = chart_df[(chart_df.Date_Time.apply(lambda x: x.date()) >= date_of_trade.date()) &
      (chart_df.Date_Time.apply(lambda x: x.date()) < date_of_trade.date().replace(day=date_of_trade.day+1))]

In [ ]:
func = lambda x: x.date()
chart_df

In [58]:
chart_df.Date_Time >= date_of_trade

TypeError: <class 'datetime.date'> type object 2018-08-06

In [74]:
trace = go.Candlestick(x=needed_chart.Date_Time,
                       open=needed_chart.Open,
                       high=needed_chart.High,
                       low=needed_chart.Low,
                       close=needed_chart.Close)
data = [trace]
py.iplot(data)